# Neural net

In [1]:
import pandas as pd
import numpy as np
import bq_helper
from bq_helper import BigQueryHelper
# https://www.kaggle.com/sohier/introduction-to-the-bq-helper-package
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Layer, Dense, Dropout, Bidirectional, Embedding, LSTM, GRU, BatchNormalization, TextVectorization, Flatten
from tensorflow.keras.optimizers import Adam
genomes = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="genomics_cannabis")

To swiftly go through the dataset:

In [2]:
bq_assistant = BigQueryHelper("bigquery-public-data", "genomics_cannabis")
bq_assistant.list_tables()

['MNPR01_201703',
 'MNPR01_reference_201703',
 'MNPR01_transcriptome_201703',
 'cs10_gff',
 'cs3k_project_info',
 'cs3k_vcf_cs10_dv090',
 'sample_info_201703']

In [3]:
query = """SELECT
  variant_id, alternate_bases, quality, type, 
  AB, ABP, AC, AF, AN, AO, DP, DPB, DPRA, EPP, 
  EPPR, GTI, MQM, MQMR, NS, NUMALT, ODDS, PAO,
  PQA, PQR, PRO, QA, QR, RO, RPL, RPP, RPPR, 
  RPR, RUN, SAF, SAP, SAR, SRF, SRP, SRR
FROM
  `bigquery-public-data.genomics_cannabis.MNPR01_201703` v
limit 1000000;"""
response = genomes.query_to_pandas_safe(query, max_gb_scanned=100)
response.head(10)

C:\Users\sharm\AppData\Roaming\Python\Python311\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,variant_id,alternate_bases,quality,type,AB,ABP,AC,AF,AN,AO,...,RPP,RPPR,RPR,RUN,SAF,SAP,SAR,SRF,SRP,SRR
0,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTY4MDF8Z2J8TU5QUj...,[TCGAAG],24.77870,[snp],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[0],[5.18177],[1],0,0.0,0
1,CKXG8eKP9qOf8wESIGdpfDEwOTg0ODk3ODZ8Z2J8TU5QUj...,[TTTTGC],17.87680,[mnp],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[1],[5.18177],[0],0,0.0,0
2,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTY3OTJ8Z2J8TU5QUj...,[TTCAATTTGT],29.52050,[complex],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[1],[5.18177],[0],0,0.0,0
3,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTU3NTR8Z2J8TU5QUj...,[ATAG],3.77924,[snp],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[1],[1],[0],[5.18177],[1],0,0.0,0
4,CKXG8eKP9qOf8wESIGdpfDEwOTg0ODk1NjR8Z2J8TU5QUj...,[TAAA],24.89040,[snp],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[0],[5.18177],[1],0,0.0,0
5,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTMwNDB8Z2J8TU5QUj...,[GAATTA],21.51740,[complex],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[1],[1],[0],[5.18177],[1],0,0.0,0
6,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTQ5MTB8Z2J8TU5QUj...,[TAAAAAAT],28.77160,[del],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[0],[5.18177],[1],0,0.0,0
7,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTYwNTl8Z2J8TU5QUj...,[GAAAA],13.94160,[complex],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[0],[5.18177],[1],0,0.0,0
8,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTcwNzZ8Z2J8TU5QUj...,[TGGCT],10.72590,[complex],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[0],[1],[0],[5.18177],[1],0,0.0,0
9,CKXG8eKP9qOf8wESIGdpfDEwOTg0ODQ3ODB8Z2J8TU5QUj...,[GA],13.93370,[snp],[0.0],[0.0],[2],[1.0],2,[1],...,[5.18177],0.0,[1],[1],[1],[5.18177],[0],0,0.0,0


To process the DNA strands:

In [4]:
response["DNA_A"] = response["alternate_bases"].str[0].str.count('A')
response["DNA_C"] = response["alternate_bases"].str[0].str.count('C')
response["DNA_G"] = response["alternate_bases"].str[0].str.count('G')
response["DNA_T"] = response["alternate_bases"].str[0].str.count('T')

To correct protein data:

In [5]:
proteins = ["AB", "ABP", "AC", "AF", "AO", "DPRA", "EPP", "MQM", "PAO",
            "PQA", "QA", "RPL", "RPP", "type",
            "RPR", "RUN", "SAF", "SAP", "SAR", 
]

for protein in proteins:
    response[protein] = response[protein].str[0]

To dummify the types:

In [6]:
dummies = pd.get_dummies(response["type"], dtype="int")

Here, I shall define my X and y. 

In [7]:
X = pd.concat([
    response[["AB", "ABP", "AC", "AF", "AN", "AO", "DP", "DPB", "DPRA", "EPP", 
              "EPPR", "GTI", "MQM", "MQMR", "NS", "NUMALT", "ODDS", "PAO",
              "PQA", "PQR", "PRO", "QA", "QR", "RO", "RPL", "RPP", "RPPR", 
              "RPR", "RUN", "SAF", "SAP", "SAR", "SRF", "SRP", "SRR"
    ]], 
    dummies
], axis=1)
y = response[["DNA_A", "DNA_C", "DNA_G", "DNA_T"]]

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import torch
import torch.nn as nn

from tqdm.notebook import tnrange, tqdm_notebook

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=1337)

In [10]:
X.shape

(1000000, 40)

In [11]:
X_train

,AB,ABP,AC,AF,AN,AO,DP,DPB,DPRA,EPP,...,SAP,SAR,SRF,SRP,SRR,complex,del,ins,mnp,snp
999016,0.0,0.0,2,1.0,2,3,3,3.0,0,9.52472,...,9.52472,0,0,0.00000,0,0,0,0,1,0
581017,0.0,0.0,0,0.0,2,2,6,6.0,0,7.35324,...,7.35324,2,3,5.18177,1,0,0,0,0,1
46168,0.0,0.0,2,1.0,2,8,9,9.0,0,7.35324,...,20.38210,8,0,0.00000,0,0,0,0,1,0
770720,0.0,0.0,2,1.0,2,3,3,3.0,0,3.73412,...,9.52472,0,0,0.00000,0,0,0,0,0,1
5842,0.0,0.0,2,1.0,2,2,2,2.0,0,7.35324,...,3.01030,1,0,0.00000,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480729,0.0,0.0,2,1.0,2,2,2,2.0,0,3.01030,...,3.01030,1,0,0.00000,0,1,0,0,0,0
216231,0.0,0.0,2,1.0,2,7,7,7.0,0,18.21060,...,18.21060,7,0,0.00000,0,0,0,0,0,1
795880,0.0,0.0,2,1.0,2,2,3,3.0,0,3.01030,...,3.01030,1,1,5.18177,0,0,0,0,0,1
975037,0.0,0.0,2,1.0,2,2,4,4.0,0,7.35324,...,7.35324,2,0,0.00000,0,0,0,0,0,1


In [12]:
X_train = torch.tensor(X_train.to_numpy(dtype='float32'), dtype=torch.float32)
X_test = torch.tensor(X_test.to_numpy(dtype='float32'), dtype=torch.float32)

y_train = torch.tensor(y_train.to_numpy(dtype='float32'), dtype=torch.long)
y_test = torch.tensor(y_test.to_numpy(dtype='float32'), dtype=torch.long)

In [13]:
x = X_train[0:8, :]
target = y_train[0:8].to(torch.float32)

neural_net = nn.Sequential(nn.Linear(40, 4),
                           nn.ReLU(),
                           nn.Linear(4, 1))
mse_loss = nn.MSELoss()

optimizer = torch.optim.SGD(neural_net.parameters(), lr=0.1)

output_values = neural_net(x)
loss = mse_loss(output_values, target)

for _ in range(0, 1000):
    loss.backward()
    optimizer.step()

    new_output = neural_net(x)
    new_loss = mse_loss(new_output, target)
    print(f"Old loss: {loss}; New loss: {new_loss}")
    loss = new_loss

c:\ProgramData\anaconda3\envs\ml_env\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([8, 4])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Old loss: 62.22698974609375; New loss: 0.6210376024246216
Old loss: 0.6210376024246216; New loss: 3.4482784271240234
Old loss: 3.4482784271240234; New loss: 7.326659202575684
Old loss: 7.326659202575684; New loss: 9.308611869812012
Old loss: 9.308611869812012; New loss: 7.887849807739258
Old loss: 7.887849807739258; New loss: 4.1441545486450195
Old loss: 4.1441545486450195; New loss: 0.9227334260940552
Old loss: 0.9227334260940552; New loss: 0.6718669533729553
Old loss: 0.6718669533729553; New loss: 3.582214117050171
Old loss: 3.582214117050171; New loss: 7.441909313201904
Old loss: 7.441909313201904; New loss: 9.317584991455078
Old loss: 9.317584991455078; New loss: 7.783727645874023
Old loss: 7.783727645874023; New loss: 4.006068706512451
Old loss: 4.006068706512451; New loss: 0.8556289672851562
Old loss: 0.8556289672851562; New loss: 0.7267420291900635
Old loss: 0.7267420291900635; New loss: 3.717362642288208
Old loss: 3.717362642288208; New loss: 7.554618835449219
Old loss: 7.55461

This is a low amount of loss. 